## Neural Networks classification with Pytorch

For readers interested in a more detailed exploration of PyTorch, we recommend referring to the official PyTorch website and its extensive documentation and tutorials (https://pytorch.org/tutorials/).

In [1]:
# We first need to import torch
# If you dont have torch, visit https://pytorch.org/get-started/locally/ and go through the setup steps

# Import torch and check version
import torch
print(torch.__version__)

2.2.0+cpu


In [2]:
# We can check if cuda is available on our device and enable it
# Otherwise we will work on cpu
if torch.cuda.is_available():
    device = "cuda" # Use NVIDIA GPU (if available)
elif torch.backends.mps.is_available():
    device = "mps" # Use Apple Silicon GPU (if available)
else:
    device = "cpu" # Default to CPU if no GPU is available

In [3]:
device

'cpu'

In [4]:
import pandas as pd

# Load the dataset for the multiclass classification
df = pd.read_csv('sheep_data.csv')


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split into features and labels
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values


In [6]:
X.shape, y.shape

((64626, 82), (64626,))

In [7]:
from sklearn.preprocessing import LabelEncoder

#'y' is our categorical labels array
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)
encoded_labels

array([0, 0, 0, ..., 4, 4, 4])

In [8]:
# Split the data 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    encoded_labels, 
                                                    test_size=0.2, 
                                                    random_state=42) 
len(X_train), len(X_test), len(y_train), len(y_test)

(51700, 12926, 51700, 12926)

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and testing data
X_train= scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [10]:
# Turning training the data into tensors
X_train = torch.from_numpy(X_train).type(torch.float)
y_train = torch.from_numpy(y_train).type(torch.long)

In [11]:
# Turning test data into tensors
X_test = torch.from_numpy(X_test).type(torch.float)
y_test = torch.from_numpy(y_test).type(torch.long)

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([51700, 82]),
 torch.Size([51700]),
 torch.Size([12926, 82]),
 torch.Size([12926]))

In [13]:
import torch.nn as nn

class MulticlassClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MulticlassClassifier, self).__init__()
        # Define the layers of the neural network
        self.fc1 = nn.Linear(input_size, 64)  # First linear layer
        self.fc2 = nn.Linear(64, 128)         # Second linear layer
        self.fc3 = nn.Linear(128, 64)         # Third linear layer
        self.fc4 = nn.Linear(64, num_classes) # Output layer

        # Activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Forward pass through the network
        x = self.relu(self.fc1(x))  # Activation function after first layer
        x = self.relu(self.fc2(x))  # Activation function after second layer
        x = self.relu(self.fc3(x))  # Activation function after third layer
        x = self.fc4(x)             # No activation function in the output layer
        return x


In [14]:
num_features = 82  
num_classes = 5   

multiclass_model = MulticlassClassifier(num_features, num_classes).to(device)

In [15]:
import torch.optim as optim

# loss and optimizer function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(multiclass_model.parameters(), lr=0.001)


In [16]:
import torch

# Assuming multiclass_model, criterion (CrossEntropyLoss), optimizer are defined
# Also assuming X_train, y_train, X_test, y_test are your datasets

torch.manual_seed(42)

# Set number of epochs
epochs = 1000

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    # Train the model
    multiclass_model.train()

    # Forward pass
    y_log = multiclass_model(X_train)
    # Convert logits to prediction labels for accuracy calculation
    y_pred = torch.argmax(y_log, dim=1)

    # loss and Accuracy
    loss = criterion(y_log, y_train) 
    correct_train = (y_pred == y_train).sum().item()
    train_accuracy = 100 * correct_train / len(y_train)

    # Zero gradients
    optimizer.zero_grad()

    # Perform backpropagation
    loss.backward()

    # Optimizer step
    optimizer.step()

    #Testing the model using .eval()
    multiclass_model.eval()
    with torch.no_grad():
        # Forward pass on test data
        test_log = multiclass_model(X_test)
        test_pred = torch.argmax(test_log, dim=1)

        # Test loss and test accuracy
        test_loss = criterion(test_log, y_test)
        correct_test = (test_pred == y_test).sum().item()
        test_accuracy = 100 * correct_test / len(y_test)

    # Print training status
    if epoch % 10 == 0:
        print(f"At Epoch: {epoch} | Training Loss: {loss:.5f}, Training Accuracy: {train_accuracy:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_accuracy:.2f}%")


At Epoch: 0 | Training Loss: 1.55996, Training Accuracy: 52.95% | Test Loss: 1.54117, Test Accuracy: 56.60%
At Epoch: 10 | Training Loss: 1.29415, Training Accuracy: 63.51% | Test Loss: 1.25302, Test Accuracy: 63.65%
At Epoch: 20 | Training Loss: 0.82347, Training Accuracy: 65.54% | Test Loss: 0.78344, Test Accuracy: 66.17%
At Epoch: 30 | Training Loss: 0.56536, Training Accuracy: 76.82% | Test Loss: 0.55207, Test Accuracy: 77.46%
At Epoch: 40 | Training Loss: 0.46899, Training Accuracy: 79.15% | Test Loss: 0.46420, Test Accuracy: 79.56%
At Epoch: 50 | Training Loss: 0.41104, Training Accuracy: 81.37% | Test Loss: 0.40861, Test Accuracy: 81.67%
At Epoch: 60 | Training Loss: 0.36694, Training Accuracy: 83.42% | Test Loss: 0.36500, Test Accuracy: 83.56%
At Epoch: 70 | Training Loss: 0.33154, Training Accuracy: 85.44% | Test Loss: 0.33061, Test Accuracy: 85.16%
At Epoch: 80 | Training Loss: 0.30091, Training Accuracy: 87.16% | Test Loss: 0.29981, Test Accuracy: 86.85%
At Epoch: 90 | Train

At Epoch: 750 | Training Loss: 0.02396, Training Accuracy: 99.30% | Test Loss: 0.08538, Test Accuracy: 97.29%
At Epoch: 760 | Training Loss: 0.02360, Training Accuracy: 99.30% | Test Loss: 0.08617, Test Accuracy: 97.28%
At Epoch: 770 | Training Loss: 0.02933, Training Accuracy: 98.95% | Test Loss: 0.09279, Test Accuracy: 97.07%
At Epoch: 780 | Training Loss: 0.02387, Training Accuracy: 99.27% | Test Loss: 0.08890, Test Accuracy: 97.25%
At Epoch: 790 | Training Loss: 0.02263, Training Accuracy: 99.32% | Test Loss: 0.08744, Test Accuracy: 97.31%
At Epoch: 800 | Training Loss: 0.02181, Training Accuracy: 99.36% | Test Loss: 0.08683, Test Accuracy: 97.26%
At Epoch: 810 | Training Loss: 0.02130, Training Accuracy: 99.38% | Test Loss: 0.08647, Test Accuracy: 97.29%
At Epoch: 820 | Training Loss: 0.02091, Training Accuracy: 99.41% | Test Loss: 0.08652, Test Accuracy: 97.32%
At Epoch: 830 | Training Loss: 0.02049, Training Accuracy: 99.40% | Test Loss: 0.08721, Test Accuracy: 97.31%
At Epoch: 

In [17]:
# Set model to evaluation mode
multiclass_model.eval()

# No gradient computation in evaluation to save memory and computations
with torch.no_grad():
    correct = 0
    total = 0
    outputs = multiclass_model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    total = y_test.size(0)
    correct = (predicted == y_test).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test dataset: {accuracy}%')

Accuracy of the model on the test dataset: 97.44700603434937%


In [18]:
from sklearn.metrics import classification_report

# Retrieve class names from label encoder
class_names = label_encoder.classes_

# Set model to evaluation mode
multiclass_model.eval()

# No gradient computation in evaluation to save memory and computations
with torch.no_grad():
    outputs = multiclass_model(X_test)
    _, predicted = torch.max(outputs.data, 1)

# Convert the tensors back to numpy arrays for sklearn compatibility
y_test_np = y_test.cpu().numpy()
predicted_np = predicted.cpu().numpy()

# Generate the classification report
report = classification_report(y_test_np, predicted_np, target_names=class_names)

print(report)

              precision    recall  f1-score   support

     grazing       1.00      1.00      1.00      3186
     resting       0.97      0.97      0.97      5568
  scratching       0.92      0.90      0.91       116
    standing       0.95      0.94      0.95      2800
     walking       0.99      1.00      0.99      1256

    accuracy                           0.97     12926
   macro avg       0.97      0.96      0.96     12926
weighted avg       0.97      0.97      0.97     12926

